<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Export-demux-reads" data-toc-modified-id="Export-demux-reads-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Export demux reads</a></span><ul class="toc-item"><li><span><a href="#Georg" data-toc-modified-id="Georg-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Georg</a></span></li><li><span><a href="#Tony" data-toc-modified-id="Tony-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Tony</a></span></li></ul></li><li><span><a href="#Copying-reads-to-/tmp/global/" data-toc-modified-id="Copying-reads-to-/tmp/global/-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Copying reads to /tmp/global/</a></span></li><li><span><a href="#Create-csv-for-local-SRA-upload" data-toc-modified-id="Create-csv-for-local-SRA-upload-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Create csv for local SRA upload</a></span><ul class="toc-item"><li><span><a href="#load/join" data-toc-modified-id="load/join-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>load/join</a></span></li><li><span><a href="#formatting" data-toc-modified-id="formatting-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>formatting</a></span></li><li><span><a href="#Writing-table" data-toc-modified-id="Writing-table-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Writing table</a></span></li></ul></li><li><span><a href="#Notes" data-toc-modified-id="Notes-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Notes</a></span></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Add 16S amplicon data (515F-806R) to local SRA, which will help with ENA submission

# Var

In [48]:
work_dir = '/tmp/global2/nyoungblut/Georg_animal_gut_16s/'

# demux dirs
georg_demux_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/georg/'
tony_demux_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/tony/'

# physeq obj. of processed 16S samples
physeq_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/phyloseq/physeq_r5k.RDS'

# params
conda_env = 'qiime2-2018.6'

# Init

In [17]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(phyloseq)
source('/ebio/abt3_projects/Georg_animal_feces/code/init.R')

In [49]:
make_dir(work_dir)

Directory already exists: /tmp/global2/nyoungblut/Georg_animal_gut_16s/ 


# Load

In [21]:
# sample metadata
metadata = physeq_file %>%
    readRDS %>%
    sample_data %>%
    as.matrix %>%
    as.data.frame %>%
    mutate(Sample = rownames(.)) %>%
    dplyr::select(Sample, scientific_name) %>%
    mutate(Sample = gsub('^X([0-9])', '\\1', Sample))

metadata %>% dfhead

Warning message in class(X) <- NULL:
“Setting class(x) to NULL;   result will no longer be an S4 object”

[1] 213   2


Sample,scientific_name
1.Eurasian.Beaver,Castor_fiber
101.Horse,Equus_caballus
102.Kulan,Equus_hemionus


# Export demux reads

## Georg

In [7]:
# paths
tmp_dir = file.path(georg_demux_dir, 'tmp')
make_dir(tmp_dir)

demux_file = file.path(georg_demux_dir, 'georg_demux.qza')
file.info(demux_file)

Directory already exists: /ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/georg//tmp 


,size,isdir,mode,mtime,ctime,atime,uid,gid,uname,grname
/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/georg//georg_demux.qza,4071486022,FALSE,644,2017-12-04 14:12:32,2017-12-04 14:12:32,2017-12-04 14:05:58,6354,353,nyoungblut,abt3


In [8]:
# exporting
cmd = 'qiime tools export --output-dir %s %s'
cmd = sprintf(cmd, tmp_dir, demux_file)
bash_job(cmd, conda_env)

character(0)

## Tony

In [9]:
# paths
tmp_dir = file.path(tony_demux_dir, 'tmp')
make_dir(tmp_dir)

demux_file = file.path(tony_demux_dir, 'tony_demux.qza')
file.info(demux_file)

Created directory: /ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/tony//tmp 


,size,isdir,mode,mtime,ctime,atime,uid,gid,uname,grname
/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/tony//tony_demux.qza,3706828419,FALSE,644,2017-12-04 17:03:26,2017-12-04 17:03:26,2017-12-04 16:57:25,6354,353,nyoungblut,abt3


In [10]:
# exporting
cmd = 'qiime tools export --output-dir %s %s'
cmd = sprintf(cmd, tmp_dir, demux_file)
bash_job(cmd, conda_env)

character(0)

# Copying reads to /tmp/global/

In [39]:
# georg
from_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/georg/tmp/'
to_dir = file.path(work_dir, 'georg/')
cmd = sprintf('cp -r %s %s', from_dir, to_dir)
bash_job(cmd, conda_env)

character(0)

In [40]:
# tony
from_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/tony/tmp/'
to_dir = file.path(work_dir, 'tony/')
cmd = sprintf('cp -r %s %s', from_dir, to_dir)
bash_job(cmd, conda_env)

character(0)

# Create csv for local SRA upload

**Columns:**

* filename = variable
* RunID = ?
* Lane = 1
* SampleID = variable
* Person = NickYoungblut
* Project = Georg_animal_gut
* Demux = yes
* SequencerID = ?

## load/join

In [41]:
# loading manifest: georg
manifest_georg_file = file.path(georg_demux_dir, 'tmp', 'MANIFEST')
read_dir = '/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg/'
manifest_georg = read.delim(manifest_georg_file, sep=',') %>%
    mutate(filename = file.path(read_dir, filename),
           RunID = 'georg')
manifest_georg %>% dfhead

[1] 344   4


sample.id,filename,direction,RunID
122.Wild.Boar,/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg//122.Wild.Boar_1_L001_R1_001.fastq.gz,forward,georg
122.Wild.Boar,/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg//122.Wild.Boar_1_L001_R2_001.fastq.gz,reverse,georg
158.White.tailed.Eagle,/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg//158.White.tailed.Eagle_2_L001_R1_001.fastq.gz,forward,georg


In [42]:
# loading manifest: tony
manifest_tony_file = file.path(tony_demux_dir, 'tmp', 'MANIFEST')
read_dir = '/tmp/global2/nyoungblut/Georg_animal_gut_16s/tony/'
manifest_tony = read.delim(manifest_tony_file, sep=',') %>%
    mutate(filename = file.path(read_dir, filename),
           RunID = 'tony')
manifest_tony %>% dfhead

[1] 308   4


sample.id,filename,direction,RunID
322.Red.Deer,/tmp/global2/nyoungblut/Georg_animal_gut_16s/tony//322.Red.Deer_1_L001_R1_001.fastq.gz,forward,tony
322.Red.Deer,/tmp/global2/nyoungblut/Georg_animal_gut_16s/tony//322.Red.Deer_1_L001_R2_001.fastq.gz,reverse,tony
215.Goose,/tmp/global2/nyoungblut/Georg_animal_gut_16s/tony//215.Goose_2_L001_R1_001.fastq.gz,forward,tony


In [43]:
# joining manifest files
manifest = rbind(manifest_georg, manifest_tony)
manifest %>% dfhead

[1] 652   4


sample.id,filename,direction,RunID
122.Wild.Boar,/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg//122.Wild.Boar_1_L001_R1_001.fastq.gz,forward,georg
122.Wild.Boar,/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg//122.Wild.Boar_1_L001_R2_001.fastq.gz,reverse,georg
158.White.tailed.Eagle,/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg//158.White.tailed.Eagle_2_L001_R1_001.fastq.gz,forward,georg


In [44]:
# overlap with metadata
setdiff(metadata$Sample, manifest$sample.id) %>% length %>% print
setdiff(manifest$sample.id, metadata$Sample) %>% length %>% print

[1] 0
[1] 106


In [45]:
# joining with metadata
manifest = manifest %>%
    inner_join(metadata, c('sample.id'='Sample'))

manifest %>% dfhead

Warning message:
“Column `sample.id`/`Sample` joining factor and character vector, coercing into character vector”

[1] 434   5


sample.id,filename,direction,RunID,scientific_name
122.Wild.Boar,/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg//122.Wild.Boar_1_L001_R1_001.fastq.gz,forward,georg,Sus_scrofa
122.Wild.Boar,/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg//122.Wild.Boar_1_L001_R2_001.fastq.gz,reverse,georg,Sus_scrofa
158.White.tailed.Eagle,/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg//158.White.tailed.Eagle_2_L001_R1_001.fastq.gz,forward,georg,Haliaeetus_albicilla


In [46]:
# duplicate samples?
manifest %>%
    group_by(sample.id) %>%
    summarize(n = n()) %>%
    ungroup() %>%
    .$n %>% table %>% print

manifest %>%
    group_by(sample.id) %>%
    summarize(n = n()) %>%
    ungroup() %>%
    filter(n == 4)

.
  2   4 
209   4 


sample.id,n
279.Western.Grey.Kangaroo,4
298.Horsfields.Bronze.Cuckoo,4
398.Eurasian.Reed.Warbler,4
F300.Asp,4


## formatting

In [53]:
manifest_f = manifest %>%
    mutate(Lane = 1,
           Person = 'NickYoungblut',
           Project = 'Georg_animal_gut',
           Demux = 'yes',
           SequencerID = 'M01032') %>%
    rename('SampleID' = sample.id) %>%
    dplyr::select(filename, RunID, Lane, SampleID, Person, Project, Demux, SequencerID)
           
manifest_f %>% dfhead

[1] 434   8


filename,RunID,Lane,SampleID,Person,Project,Demux,SequencerID
/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg//122.Wild.Boar_1_L001_R1_001.fastq.gz,georg,1,122.Wild.Boar,NickYoungblut,Georg_animal_gut,yes,M01032
/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg//122.Wild.Boar_1_L001_R2_001.fastq.gz,georg,1,122.Wild.Boar,NickYoungblut,Georg_animal_gut,yes,M01032
/tmp/global2/nyoungblut/Georg_animal_gut_16s/georg//158.White.tailed.Eagle_2_L001_R1_001.fastq.gz,georg,1,158.White.tailed.Eagle,NickYoungblut,Georg_animal_gut,yes,M01032


## Writing table

In [54]:
out_file = file.path(work_dir, 'localSRA_import.csv')
write.table(manifest_f, out_file, sep=',', quote=FALSE, row.names=FALSE)
cat('File written:', out_file, '\n')

File written: /tmp/global2/nyoungblut/Georg_animal_gut_16s//localSRA_import.csv 


# Notes

* Ilja Bezrukov submitted the reads to the local SRA on Oct 19
  * RunIds
    * "Georg" = 001
    * "Tony" = 002
  * No. of entries = 434
  
> Note: all "." in sampleIDs changed to "_"

# sessionInfo

In [51]:
sessionInfo()

R version 3.4.1 (2017-06-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.1 LTS

Matrix products: default
BLAS: /ebio/abt3_projects/software/miniconda3_gt4.4/envs/py3_physeq/lib/R/lib/libRblas.so
LAPACK: /ebio/abt3_projects/software/miniconda3_gt4.4/envs/py3_physeq/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] phyloseq_1.22.3 bindrcpp_0.2.2  ggplot2_3.0.0   tidyr_0.8.1    
[5] dplyr_0.7.6    

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.17        ape_5.1             lattice_0.20-34    
 [4

In [52]:
condaInfo(conda_env)

# packages in environment at /ebio/abt3_projects/software/miniconda3_gt4.4/envs/qiime2-2018.6:
#
# Name                    Version                   Build  Channel
asn1crypto                0.24.0                     py_1    conda-forge
backcall                  0.1.0                      py_0    conda-forge
bibtexparser              1.0.1                      py_1    conda-forge
bioconductor-biobase      2.38.0                 r3.4.1_0    bioconda
bioconductor-biocgenerics 0.24.0                 r3.4.1_0    bioconda
bioconductor-biocparallel 1.12.0                 r3.4.1_0    bioconda
bioconductor-biostrings   2.46.0                 r3.4.1_0    bioconda
bioconductor-dada2        1.6.0                  r3.4.1_0    bioconda
bioconductor-delayedarray 0.4.1                  r3.4.1_0    bioconda
bioconductor-genomeinfodb 1.14.0                 r3.4.1_0    bioconda
bioconductor-genomeinfodbdata 1.0.0                  r3.4.1_1    bioconda
bioconductor-genomicalignments 1.14.1                